[How to clean](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html)

[Deployment code](https://github.com/raja-surya/aws-deployment-1/blob/main/Iris-model-deployment.ipynb)


In [ ]:
# setup
%%time

import os
import boto3
import sagemaker

from sagemaker import get_execution_role

region = boto3.Session().region_name

role = get_execution_role()

In [ ]:
# create s3 bucket
bucket = sagemaker.Session().default_bucket()
bucket_path = "https://s3-{}.amazonaws.com/{}".format(region, bucket)

print(role,region,bucket,bucket_path)

In [ ]:
# installing the sklearn package
!conda install -y -c conda-forge scikit-learn

In [ ]:
# loading the model

model_file_name = "recommendation_model"

import joblib
import sklearn

model = joblib.load(model_file_name)

In [ ]:
# converting the model to tar file
!tar czvf model.tar.gz $model_file_name

In [ ]:
# upload the pretrained model to bucket
prefix ="sagemaker/demo-recommendation"

fileObject = open("model.tar.gz","rb")
key = os.path.join(prefix,model_file_name,"model.tar.gz")
print(key)
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(f

In [ ]:
# setting up for hosting
from sagemaker.amazon.amazon_estimator import get_image_uri

# container = get_image_uri(boto3.Session().region_name, "xgboost", "0.90-1")
container = get_image_uri(boto3.Session().region_name,"sklearn","1.4.0")

In [ ]:
# creating the model in sagemaker

%%time
from time import gmtime, strftime

model_name = model_file_name + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

model_url = "https://s3-{}.amazonaws.com/{}/{}".format(region, bucket, key)

sm_client = boto3.client("sagemaker")

print(model_url)

primary_container = {
    "Image": container,
    "ModelDataUrl": model_url,
}

create_model_response2 = sm_client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, PrimaryContainer=primary_container
)

print(create_model_response2["ModelArn"])

In [ ]:
from time import gmtime,strftime

endpoint_config_name = "demo-recommendation-config" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

print(endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.t2.medium",
            "InitialInstanceCount": 1,
            "InitialVariantWeight": 1,
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

In [ ]:
# create endpoint

%%time
import time

endpoint_name = "recommendationEndpoint"+ strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

In [ ]:
# testing the sagemaker model
movie = "Tiempo"

response = runtime_client.invoke_endpoint(
    EndpointName = endpoint_name,ContentType="text/csv",Body=movie
)

result = response["Body"].read().decode("ascii")

print("\n Predicted".format(result))

In [ ]:
# to delete the endpoint
sm_client.delete_endpoint(EndpointName=endpoint_name)